In [1]:
import pandas as pd
import streamlit as st


In [25]:
# df_strava = pd.read_csv('atividades_unicas.csv')
df_strava = pd.read_csv('http://145.223.30.123:8000/atividades_unicas.csv')
df_strava['athlete.fullname'] = df_strava['firstname'] + ' ' + df_strava['lastname']
df_strava['moving_time_minutes'] = round(df_strava['moving_time']/60, 2)

In [26]:
cols = ['athlete.fullname','name', 'type', 'distance',   
         'data_atual', 'distance_km', 'pace_real','moving_time_minutes'
       ]
# df_strava['moving_time_minutes'] = round(df_strava['moving_time']/60, 2)

df_strava['distance_km'] = round(df_strava['distance'] / 1000, 2)

df_strava['pace'] = df_strava['moving_time_minutes'] / df_strava['distance_km']

df_strava['avg_speed_kmh'] = round(60/df_strava['pace'], 2)


def kmh_to_min_km(speed_kmh):
    if speed_kmh > 0:  # Evitar divisão por zero
        pace = 60 / speed_kmh
        minutes = int(pace)
        seconds = int((pace - minutes) * 60)
        return f"{minutes}:{seconds:02d} min/km"
    else:
        return None  # Retorna None para velocidades inválidas

df_strava['avg_speed_kmh'] = pd.to_numeric(df_strava['avg_speed_kmh'], errors='coerce')
df_strava['pace_real'] = df_strava['avg_speed_kmh'].apply(kmh_to_min_km)

corridas = df_strava[cols]
runs = corridas.loc[corridas['type'] == 'Run' ].sort_values(by='athlete.fullname')
# Agrupar por atleta e agregar as datas em uma lista
grouped = runs.groupby("athlete.fullname").agg(
    total_quilometros=("distance", lambda x: round(x.sum() / 1000, 2)),
    quantidade_atividades=("moving_time_minutes", "count"),
    datas=("data_atual", lambda x: list(x))
).reset_index()

# Expandir as listas de datas em colunas separadas
datas_expandidas = grouped['datas'].apply(pd.Series)
datas_expandidas.columns = [f'dia - {i+1}' for i in range(datas_expandidas.shape[1])]
# Substituir NaN por um traço
datas_expandidas.fillna('-', inplace=True)
# Formatar as datas para o formato 'dd-mm-yy'
for col in datas_expandidas.columns:
    datas_expandidas[col] = pd.to_datetime(datas_expandidas[col], errors='coerce').dt.strftime('%d-%m-%y')
    datas_expandidas[col] = datas_expandidas[col].fillna('-')
    
# Concatenar as colunas expandidas com o DataFrame original
pontuacao_participantes = pd.concat([grouped.drop(columns=['datas']), datas_expandidas], axis=1)


pontuacao_participantes





,athlete.fullname,total_quilometros,quantidade_atividades,dia - 1
0,Larissa C.,4.28,1,09-01-25
1,Mara Lúcia C.,4.67,1,09-01-25
2,Renato N.,5.26,1,09-01-25
3,Sandro M.,2.96,1,09-01-25
4,Vinícius G.,8.01,1,09-01-25
